# Parser for Govinfo

In [46]:
import pandas as pd
import requests
import numpy as np
import os
import re

## Save the search result from `https://www.govinfo.gov/#advanced` as a `csv` in the same directory as this notebook. Put the file name below. 
The website allows the searched result to be saved with a maximum of 1000 rows. No documentation mentions the limit of file access. 
## Put the api_key below

In [1]:
from yaml import load, CLoader as Loader
import os

path = os.path.abspath(os.path.join(__file__, os.pardir))

api_key = load( open(os.path.join(path,'../api_keys.yaml'),
                    'rb'), Loader=Loader)

file_name = 'example.csv'

ModuleNotFoundError: No module named 'yaml'

In [34]:
search_result = pd.read_csv(file_name, skiprows = [0,1])
# search_result

In [44]:
col = ['date', 'hour', 'Title', 'Summary/Description', 'Body/Contents', 'Source “URL”', 'Metadata']
df = pd.DataFrame(columns=col, dtype = 'str')

In [47]:
def read_append(url, row, df1):
    url_api = url + api_key
    r = requests.get(url_api, allow_redirects=True)
    
    

    text = re.sub('<[^<]+>', "", str(r.content))
    
    
    new_row = [row['publishdate'], np.nan, row['title'], row['teaser'], text, url, '[gov_info]']
    df1 = pd.concat([pd.DataFrame([new_row], columns=col), df1], ignore_index=True)
    return df1

## The file will be saved as either `txt` or `xml`, with the name of its index in the csv. 
For each file, the available types are different. The .pdf files are ignored; .htm files are plain text thus saved as .txt; .xml files needs to be further parsed. 

In [48]:
for i, row in search_result.iterrows():
    # print(pd.isna(row['xmlLink']))
    if not pd.isna(row['htmlLink']):
        df = read_append(row['htmlLink'], row, df)
    elif not pd.isna(row['xmlLink']):
        df = read_append(row['xmlLink'], row, df)
        
    elif str(row['otherLink1']).endswith('htm'):
        df = read_append(row['otherLink1'], row, df)
    elif str(row['otherLink1']).endswith('xml'):
        df = read_append(row['otherLink1'], row, df)
        
    elif str(row['otherLink2']).endswith('htm'):
        df = read_append(row['otherLink2'], row, df)
    elif str(row['otherLink2']).endswith('xml'):
        df = read_append(row['otherLink2'], row, df)
    
    print('row ' + str(i) + ' completed', end = '\r')
df

,date,hour,Title,Summary/Description,Body/Contents,Source “URL”,Metadata
0,NaN,NaN,NaN,NaN,b'\n\n\n \n 26\n Internal Rev...,https://api.govinfo.gov/packages/CFR-2021-titl...,[gov_info]
1,9/23/2022,NaN,"Status of Appropriation Bills, First Session, ...",agreed to became public measure number short ...,"b'\nSenate Calendars for September 23, 2022 - ...",https://www.govinfo.gov/content/pkg/CCAL-117sc...,[gov_info]
2,9/23/2022,NaN,Bills in Conference,House asking it are to leave the papers with ...,"b""\nSenate Calendars for September 23, 2022 - ...",https://www.govinfo.gov/content/pkg/CCAL-117sc...,[gov_info]
3,9/23/2022,NaN,Subjects on the Table,"status, it is in order to move to proceed to i...","b""\nSenate Calendars for September 23, 2022 - ...",https://www.govinfo.gov/content/pkg/CCAL-117sc...,[gov_info]
4,9/23/2022,NaN,Notice of Intent to Object to Proceeding,submitted for inclusion in the Congressional R...,"b""\nSenate Calendars for September 23, 2022 - ...",https://www.govinfo.gov/content/pkg/CCAL-117sc...,[gov_info]
5,9/23/2022,NaN,General Orders,"an Jan. 6, 2021.--Read the Senator exception ...","b""\nSenate Calendars for September 23, 2022 - ...",https://www.govinfo.gov/content/pkg/CCAL-117sc...,[gov_info]
6,9/23/2022,NaN,Cross Index of General Orders Measures with Co...,13 2 S. 14 88 S. 15 191 S. 46 223 S. 82 204 S....,"b'\nSenate Calendars for September 23, 2022 - ...",https://www.govinfo.gov/content/pkg/CCAL-117sc...,[gov_info]
7,9/23/2022,NaN,Joint Committees of the Congress,"Amy Klobuchar, of Minnesota Margaret Wood Has...","b'\nSenate Calendars for September 23, 2022 - ...",https://www.govinfo.gov/content/pkg/CCAL-117sc...,[gov_info]
8,9/23/2022,NaN,"Other, Select and Special Committees",", of Washington Jon Tester, of Montana Catheri...","b'\nSenate Calendars for September 23, 2022 - ...",https://www.govinfo.gov/content/pkg/CCAL-117sc...,[gov_info]
9,9/23/2022,NaN,Committee Assignments,"of the Chairman. Debbie Stabenow, of Michigan...","b""\nSenate Calendars for September 23, 2022 - ...",https://www.govinfo.gov/content/pkg/CCAL-117sc...,[gov_info]
